In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import seaborn as sns
from warnings import filterwarnings
filterwarnings('ignore')
pd.options.display.max_columns = None
pd.options.display.max_rows = None
from sklearn.model_selection import train_test_split
import statsmodels
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.preprocessing import StandardScaler 
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import log_loss
from pandas.api.types import is_string_dtype
from sklearn.feature_selection import RFE

In [2]:
pd.set_option('display.max_columns', None) 
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None) 

In [4]:
df=pd.read_csv('final_5june.csv')

In [5]:
df.shape

(261276, 105)

In [6]:
df.head()

,Unnamed: 0,term_ 60 months,grade_B,grade_C,grade_D,grade_E,grade_F,grade_G,emp_length_between 5-9 years,emp_length_less than 5 years,home_ownership_OTHERS,home_ownership_OWN,home_ownership_RENT,verification_status_Source Verified,verification_status_Verified,purpose_debt_consolidation,purpose_others,initial_list_status_w,application_type_Joint App,hardship_flag_Y,disbursement_method_DirectPay,debt_settlement_flag_Y,region_Central,region_Eastern,region_Hawaii-Aleutian,region_Mountain,region_Pacific,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,mths_since_last_delinq,open_acc,pub_rec,revol_bal,revol_util,total_acc,out_prncp,out_prncp_inv,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_amnt,collections_12_mths_ex_med,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,loan_amnt,policy_code,months_to_issue,current_loan_status
0,0,0,0,0,0,0,0,0,1,0,0,0,1,1,0,0,1,0,0,0,0,0,0,1,0,0,0,-1.567936,-1.392435,-0.986509,-0.039362,0.892104,-0.764979,-0.745474,-0.682434,-0.365964,-1.186021,0.091159,0.616506,-0.007196,-0.007196,-1.036836,-0.246594,-0.757197,-0.666547,-0.683193,0.0,0.0,-0.402557,-0.012658,-1.525434,1.093505,0.105268,0.540924,0.137798,2.047183,-0.212204,-1.530517,-0.799333,-2.141240,0.746891,-1.506434,3.562041,0.171476,2.064590,-0.331070,0.426737,-0.968501,1.333178,-0.085251,-0.018973,1.489376,0.400316,0.851669,0.625149,-0.847759,-0.077517,0.189427,-0.472241,0.978461,-0.804103,-1.329182,-1.047065,-1.390015,2.450664,-1.033484,-1.065226,-1.219317,-0.728216,-0.025269,-0.054988,2.935668,-0.795993,0.277153,1.450938,-0.398975,-0.141946,-0.325048,1.946388,-1.269134,1.909787,-1.395629,0.0,0.241407,1.0
1,1,1,0,0,0,1,0,0,1,0,0,0,1,1,0,1,0,1,0,0,0,1,0,0,0,0,1,1.658467,0.398425,1.510002,0.351551,-0.423744,0.253107,0.388265,-0.147136,-0.365964,-0.901022,-0.471494,-0.825149,-0.007196,-0.007196,2.020670,-0.246594,2.074355,2.719636,-0.285892,0.0,0.0,-0.402557,-0.149993,-0.040616,-0.181647,0.105268,-0.296918,-0.087826,-0.046629,0.340748,-0.188041,0.084595,-0.024177,0.125057,-1.006902,-0.068834,-0.333912,0.002983,1.221994,-0.140797,-0.686323,-0.589552,-0.085251,-0.018973,-1.635345,-1.539414,0.720456,0.121236,-0.847759,0.524990,-0.966213,-0.472241,-0.521321,-0.804103,-0.263412,-1.047065,-1.390015,0.136476,-0.564859,-1.065226,-0.269476,-0.142452,-0.025269,-0.054988,-0.237952,-0.795993,0.659437,0.026743,-0.398975,-0.141946,-0.187378,1.790584,-1.090579,1.909787,0.545943,0.0,-1.827847,1.0
2,2,0,0,0,0,1,0,0,1,0,0,0,1,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0.865089,1.199525,-0.639771,0.648573,-0.423744,-0.764979,-0.367561,0.566594,-0.365964,-0.202660,-0.306505,1.888555,-0.007196,-0.007196,1.933926,2.261176,-0.757197,-0.666547,-0.075377,0.0,0.0,1.715405,-0.380319,-0.040616,-0.181647,0.105268,-0.296918,-0.087826,-0.046629,0.340748,-0.188041,0.084595,-0.024177,0.125057,-0.039453,-0.068834,-0.333912,0.002983,1.221994,-0.580517,-0.585271,0.718052,-0.085251,-0.018973,1.352527,-0.616370,0.195605,-0.508655,-0.847759,-0.412243,-0.388393,1.488087,-0.521321,0.143087,-0.263412,-0.252556,-0.908949,2.450664,0.606701,-0.439615,-0.269476,0.638565,-0.025269,-0.054988,-0.237952,-0.190302,0.103387,0.026743,-0.398975,-0.141946,-0.339977,0.968290,-0.205921,1.305473,0.671759,0.0,-0

In [8]:
df.drop(columns='Unnamed: 0',axis=1,inplace=True)

In [9]:
df.shape

(261276, 104)

In [10]:
df.drop(columns=['collections_12_mths_ex_med','acc_now_delinq','policy_code'],axis=1,inplace=True)

In [11]:
df.drop(columns=['installment','out_prncp','collection_recovery_fee','avg_cur_bal','bc_util','num_actv_rev_tl','num_bc_sats','num_op_rev_tl','num_rev_accts','num_rev_tl_bal_gt_0','num_sats','percent_bc_gt_75','tot_hi_cred_lim','total_bc_limit','total_il_high_credit_limit','months_to_issue'],axis=1,inplace=True)

In [12]:
df.shape

(261276, 85)

In [13]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Function to calculate VIF
def calculate_vif(df):
    vif = pd.DataFrame()
    vif["Variable"] = df.columns
    vif["VIF"] = [variance_inflation_factor(df.values, i) for i in range(df.shape[1])]
    return vif


In [14]:
vif = calculate_vif(df)


In [15]:
print(vif)

                               Variable        VIF
0                       term_ 60 months   2.686159
1                               grade_B   5.147303
2                               grade_C  13.067174
3                               grade_D  16.608205
4                               grade_E  14.490486
5                               grade_F   7.425577
6                               grade_G   3.044422
7          emp_length_between 5-9 years   1.687541
8          emp_length_less than 5 years   2.228727
9                 home_ownership_OTHERS   1.002236
10                   home_ownership_OWN   1.416655
11                  home_ownership_RENT   3.582592
12  verification_status_Source Verified   2.829687
13         verification_status_Verified   2.817729
14           purpose_debt_consolidation   4.323386
15                       purpose_others   2.299557
16                initial_list_status_w   2.727885
17           application_type_Joint App   1.117182
18                      hardshi

In [16]:
high_vif_columns = vif[vif["VIF"] > 10]["Variable"]
df = df.drop(columns=high_vif_columns)

In [18]:
df.shape

(261276, 77)

In [19]:
df.head()

,term_ 60 months,grade_B,grade_F,grade_G,emp_length_between 5-9 years,emp_length_less than 5 years,home_ownership_OTHERS,home_ownership_OWN,home_ownership_RENT,verification_status_Source Verified,verification_status_Verified,purpose_debt_consolidation,purpose_others,initial_list_status_w,application_type_Joint App,hardship_flag_Y,disbursement_method_DirectPay,debt_settlement_flag_Y,region_Hawaii-Aleutian,region_Mountain,annual_inc,dti,delinq_2yrs,inq_last_6mths,mths_since_last_delinq,open_acc,pub_rec,revol_bal,revol_util,total_acc,out_prncp_inv,total_rec_int,total_rec_late_fee,recoveries,last_pymnt_amnt,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,bc_open_to_buy,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_bc_tl,num_il_tl,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,pub_rec_bankruptcies,tax_liens,total_bal_ex_mort,loan_amnt
0,0,0,0,0,1,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,-0.986509,-0.039362,0.892104,-0.764979,-0.745474,-0.682434,-0.365964,-1.186021,0.091159,0.616506,-0.007196,-1.036836,-0.246594,-0.757197,-0.683193,-0.402557,-0.012658,-1.525434,1.093505,0.105268,0.540924,0.137798,2.047183,-0.212204,-1.530517,-0.799333,-2.141240,0.746891,-1.506434,3.562041,0.171476,2.064590,-0.331070,-0.968501,-0.085251,-0.018973,1.489376,0.400316,0.851669,0.625149,-0.847759,-0.077517,0.189427,-0.472241,0.978461,-0.804103,-1.390015,2.450664,-0.025269,-0.054988,2.935668,-0.795993,0.277153,-0.398975,-0.141946,1.946388,-1.395629
1,1,0,0,0,1,0,0,0,1,1,0,1,0,1,0,0,0,1,0,0,1.510002,0.351551,-0.423744,0.253107,0.388265,-0.147136,-0.365964,-0.901022,-0.471494,-0.825149,-0.007196,2.020670,-0.246594,2.074355,-0.285892,-0.402557,-0.149993,-0.040616,-0.181647,0.105268,-0.296918,-0.087826,-0.046629,0.340748,-0.188041,0.084595,-0.024177,0.125057,-1.006902,-0.068834,-0.333912,0.002983,1.221994,-0.686323,-0.085251,-0.018973,-1.635345,-1.539414,0.720456,0.121236,-0.847759,0.524990,-0.966213,-0.472241,-0.521321,-0.804103,-1.390015,0.136476,-0.025269,-0.054988,-0.237952,-0.795993,0.659437,-0.398975,-0.141946,1.790584,0.545943
2,0,0,0,0,1,0,0,0,1,0,1,1,0,0,0,0,0,0,0,0,-0.639771,0.648573,-0.423744,-0.764979,-0.367561,0.566594,-0.365964,-0.202660,-0.306505,1.888555,-0.007196,1.933926,2.261176,-0.757197,-0.075377,1.715405,-0.380319,-0.040616,-0.181647,0.105268,-0.296918,-0.087826,-0.046629,0.340748,-0.188041,0.084595,-0.024177,0.125057,-0.039453,-0.068834,-0.333912,0.002983,1.221994,-0.585271,-0.085251,-0.018973,1.352527,-0.616370,0.195605,-0.508655,-0.847759,-0.412243,-0.388393,1.488087,-0.521321,0.143087,-0.908949,2.450664,-0.025269,-0.054988,-0.237952,-0.190302,0.103387,-0.398975,-0.141946,0.968290,0.671759
3,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0.053704,-1.722182,-0.423744,3.307367,0.388265,-1.753029,-0.365964,-1.305674,-2.202602,0.362096,-0.007196,0.160809,-0.246594,-0.747537,-0.335357,-0.402557,-0.441812,-0.040616,-0.181647,0.105268,-0.296918,-0.087826,-0.046629,0.340748,-0.188041,0.084595,-0.024177,0.125057,-0.229149,-0.068834,-0.333912,0.002983,0.123387,-0.391283,-0.085251,-0.018973,0.166501,-0.121405,-0.329246,-0.256699,-0.221818,-0.345298,-0.195786,-0.472241,-0.521321,-0.330508,-0.187350,-0.194123,-0.025269,-0.054988,-0.237952,-0.190302,0.659437,-0.398975,-0.141946,-0.220710,1.117834
4,0,0,0,0,0,1,0,0,1,1,0,0,0,1,0,0,0,0,0,0,-0.847814,-0.953899,-0.423744,-0.764979,0.703192,-1.217732,-0.365964,-0.493590,0.755343,-0.909952,-0.007196,-0.823654,-0.246594,-0.757197,0.906318,-0.402557,-1.003996,-0.040616,-0.181647,0.105268,-0.296918,-0.087826,-0.046629,0.340748,-0.188041,0.084595,-0.024177,0.125057,-0.880438,-0.068834,-0.333912,0.002983,0.057196,-0.909248,-0.08

In [20]:
df2=pd.read_csv('final_5june.csv')

In [21]:
df2.head()

,Unnamed: 0,term_ 60 months,grade_B,grade_C,grade_D,grade_E,grade_F,grade_G,emp_length_between 5-9 years,emp_length_less than 5 years,home_ownership_OTHERS,home_ownership_OWN,home_ownership_RENT,verification_status_Source Verified,verification_status_Verified,purpose_debt_consolidation,purpose_others,initial_list_status_w,application_type_Joint App,hardship_flag_Y,disbursement_method_DirectPay,debt_settlement_flag_Y,region_Central,region_Eastern,region_Hawaii-Aleutian,region_Mountain,region_Pacific,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,mths_since_last_delinq,open_acc,pub_rec,revol_bal,revol_util,total_acc,out_prncp,out_prncp_inv,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_amnt,collections_12_mths_ex_med,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,loan_amnt,policy_code,months_to_issue,current_loan_status
0,0,0,0,0,0,0,0,0,1,0,0,0,1,1,0,0,1,0,0,0,0,0,0,1,0,0,0,-1.567936,-1.392435,-0.986509,-0.039362,0.892104,-0.764979,-0.745474,-0.682434,-0.365964,-1.186021,0.091159,0.616506,-0.007196,-0.007196,-1.036836,-0.246594,-0.757197,-0.666547,-0.683193,0.0,0.0,-0.402557,-0.012658,-1.525434,1.093505,0.105268,0.540924,0.137798,2.047183,-0.212204,-1.530517,-0.799333,-2.141240,0.746891,-1.506434,3.562041,0.171476,2.064590,-0.331070,0.426737,-0.968501,1.333178,-0.085251,-0.018973,1.489376,0.400316,0.851669,0.625149,-0.847759,-0.077517,0.189427,-0.472241,0.978461,-0.804103,-1.329182,-1.047065,-1.390015,2.450664,-1.033484,-1.065226,-1.219317,-0.728216,-0.025269,-0.054988,2.935668,-0.795993,0.277153,1.450938,-0.398975,-0.141946,-0.325048,1.946388,-1.269134,1.909787,-1.395629,0.0,0.241407,1.0
1,1,1,0,0,0,1,0,0,1,0,0,0,1,1,0,1,0,1,0,0,0,1,0,0,0,0,1,1.658467,0.398425,1.510002,0.351551,-0.423744,0.253107,0.388265,-0.147136,-0.365964,-0.901022,-0.471494,-0.825149,-0.007196,-0.007196,2.020670,-0.246594,2.074355,2.719636,-0.285892,0.0,0.0,-0.402557,-0.149993,-0.040616,-0.181647,0.105268,-0.296918,-0.087826,-0.046629,0.340748,-0.188041,0.084595,-0.024177,0.125057,-1.006902,-0.068834,-0.333912,0.002983,1.221994,-0.140797,-0.686323,-0.589552,-0.085251,-0.018973,-1.635345,-1.539414,0.720456,0.121236,-0.847759,0.524990,-0.966213,-0.472241,-0.521321,-0.804103,-0.263412,-1.047065,-1.390015,0.136476,-0.564859,-1.065226,-0.269476,-0.142452,-0.025269,-0.054988,-0.237952,-0.795993,0.659437,0.026743,-0.398975,-0.141946,-0.187378,1.790584,-1.090579,1.909787,0.545943,0.0,-1.827847,1.0
2,2,0,0,0,0,1,0,0,1,0,0,0,1,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0.865089,1.199525,-0.639771,0.648573,-0.423744,-0.764979,-0.367561,0.566594,-0.365964,-0.202660,-0.306505,1.888555,-0.007196,-0.007196,1.933926,2.261176,-0.757197,-0.666547,-0.075377,0.0,0.0,1.715405,-0.380319,-0.040616,-0.181647,0.105268,-0.296918,-0.087826,-0.046629,0.340748,-0.188041,0.084595,-0.024177,0.125057,-0.039453,-0.068834,-0.333912,0.002983,1.221994,-0.580517,-0.585271,0.718052,-0.085251,-0.018973,1.352527,-0.616370,0.195605,-0.508655,-0.847759,-0.412243,-0.388393,1.488087,-0.521321,0.143087,-0.263412,-0.252556,-0.908949,2.450664,0.606701,-0.439615,-0.269476,0.638565,-0.025269,-0.054988,-0.237952,-0.190302,0.103387,0.026743,-0.398975,-0.141946,-0.339977,0.968290,-0.205921,1.305473,0.671759,0.0,-0

In [22]:
df['current_loan_status']=df2.current_loan_status

In [23]:
df.head()

,term_ 60 months,grade_B,grade_F,grade_G,emp_length_between 5-9 years,emp_length_less than 5 years,home_ownership_OTHERS,home_ownership_OWN,home_ownership_RENT,verification_status_Source Verified,verification_status_Verified,purpose_debt_consolidation,purpose_others,initial_list_status_w,application_type_Joint App,hardship_flag_Y,disbursement_method_DirectPay,debt_settlement_flag_Y,region_Hawaii-Aleutian,region_Mountain,annual_inc,dti,delinq_2yrs,inq_last_6mths,mths_since_last_delinq,open_acc,pub_rec,revol_bal,revol_util,total_acc,out_prncp_inv,total_rec_int,total_rec_late_fee,recoveries,last_pymnt_amnt,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,bc_open_to_buy,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_bc_tl,num_il_tl,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,pub_rec_bankruptcies,tax_liens,total_bal_ex_mort,loan_amnt,current_loan_status
0,0,0,0,0,1,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,-0.986509,-0.039362,0.892104,-0.764979,-0.745474,-0.682434,-0.365964,-1.186021,0.091159,0.616506,-0.007196,-1.036836,-0.246594,-0.757197,-0.683193,-0.402557,-0.012658,-1.525434,1.093505,0.105268,0.540924,0.137798,2.047183,-0.212204,-1.530517,-0.799333,-2.141240,0.746891,-1.506434,3.562041,0.171476,2.064590,-0.331070,-0.968501,-0.085251,-0.018973,1.489376,0.400316,0.851669,0.625149,-0.847759,-0.077517,0.189427,-0.472241,0.978461,-0.804103,-1.390015,2.450664,-0.025269,-0.054988,2.935668,-0.795993,0.277153,-0.398975,-0.141946,1.946388,-1.395629,1.0
1,1,0,0,0,1,0,0,0,1,1,0,1,0,1,0,0,0,1,0,0,1.510002,0.351551,-0.423744,0.253107,0.388265,-0.147136,-0.365964,-0.901022,-0.471494,-0.825149,-0.007196,2.020670,-0.246594,2.074355,-0.285892,-0.402557,-0.149993,-0.040616,-0.181647,0.105268,-0.296918,-0.087826,-0.046629,0.340748,-0.188041,0.084595,-0.024177,0.125057,-1.006902,-0.068834,-0.333912,0.002983,1.221994,-0.686323,-0.085251,-0.018973,-1.635345,-1.539414,0.720456,0.121236,-0.847759,0.524990,-0.966213,-0.472241,-0.521321,-0.804103,-1.390015,0.136476,-0.025269,-0.054988,-0.237952,-0.795993,0.659437,-0.398975,-0.141946,1.790584,0.545943,1.0
2,0,0,0,0,1,0,0,0,1,0,1,1,0,0,0,0,0,0,0,0,-0.639771,0.648573,-0.423744,-0.764979,-0.367561,0.566594,-0.365964,-0.202660,-0.306505,1.888555,-0.007196,1.933926,2.261176,-0.757197,-0.075377,1.715405,-0.380319,-0.040616,-0.181647,0.105268,-0.296918,-0.087826,-0.046629,0.340748,-0.188041,0.084595,-0.024177,0.125057,-0.039453,-0.068834,-0.333912,0.002983,1.221994,-0.585271,-0.085251,-0.018973,1.352527,-0.616370,0.195605,-0.508655,-0.847759,-0.412243,-0.388393,1.488087,-0.521321,0.143087,-0.908949,2.450664,-0.025269,-0.054988,-0.237952,-0.190302,0.103387,-0.398975,-0.141946,0.968290,0.671759,1.0
3,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0.053704,-1.722182,-0.423744,3.307367,0.388265,-1.753029,-0.365964,-1.305674,-2.202602,0.362096,-0.007196,0.160809,-0.246594,-0.747537,-0.335357,-0.402557,-0.441812,-0.040616,-0.181647,0.105268,-0.296918,-0.087826,-0.046629,0.340748,-0.188041,0.084595,-0.024177,0.125057,-0.229149,-0.068834,-0.333912,0.002983,0.123387,-0.391283,-0.085251,-0.018973,0.166501,-0.121405,-0.329246,-0.256699,-0.221818,-0.345298,-0.195786,-0.472241,-0.521321,-0.330508,-0.187350,-0.194123,-0.025269,-0.054988,-0.237952,-0.190302,0.659437,-0.398975,-0.141946,-0.220710,1.117834,1.0
4,0,0,0,0,0,1,0,0,1,1,0,0,0,1,0,0,0,0,0,0,-0.847814,-0.953899,-0.423744,-0.764979,0.703192,-1.217732,-0.365964,-0.493590,0.755343,-0.909952,-0.007196,-0.823654,-0.246594,-0.757197,0.906318,-0.402557,-1.003996,-0.040616,-0.181647,0.105268,-0.296918,-0.087826,-0.046629,0.340748,-0.188041,0.084595,-0.024177,0.125057,-0.880438,-0.068834,-0.3339

In [24]:
df.shape

(261276, 78)

In [26]:
nunique = df.nunique()
constant_columns = nunique[nunique == 1].index
df = df.drop(columns=constant_columns)


In [27]:
df.shape

(261276, 78)

In [28]:
X=df.iloc[:,:77]
y=df.current_loan_status

In [29]:
X.shape

(261276, 77)

In [31]:
X.head()

,term_ 60 months,grade_B,grade_F,grade_G,emp_length_between 5-9 years,emp_length_less than 5 years,home_ownership_OTHERS,home_ownership_OWN,home_ownership_RENT,verification_status_Source Verified,verification_status_Verified,purpose_debt_consolidation,purpose_others,initial_list_status_w,application_type_Joint App,hardship_flag_Y,disbursement_method_DirectPay,debt_settlement_flag_Y,region_Hawaii-Aleutian,region_Mountain,annual_inc,dti,delinq_2yrs,inq_last_6mths,mths_since_last_delinq,open_acc,pub_rec,revol_bal,revol_util,total_acc,out_prncp_inv,total_rec_int,total_rec_late_fee,recoveries,last_pymnt_amnt,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,bc_open_to_buy,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_bc_tl,num_il_tl,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,pub_rec_bankruptcies,tax_liens,total_bal_ex_mort,loan_amnt
0,0,0,0,0,1,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,-0.986509,-0.039362,0.892104,-0.764979,-0.745474,-0.682434,-0.365964,-1.186021,0.091159,0.616506,-0.007196,-1.036836,-0.246594,-0.757197,-0.683193,-0.402557,-0.012658,-1.525434,1.093505,0.105268,0.540924,0.137798,2.047183,-0.212204,-1.530517,-0.799333,-2.141240,0.746891,-1.506434,3.562041,0.171476,2.064590,-0.331070,-0.968501,-0.085251,-0.018973,1.489376,0.400316,0.851669,0.625149,-0.847759,-0.077517,0.189427,-0.472241,0.978461,-0.804103,-1.390015,2.450664,-0.025269,-0.054988,2.935668,-0.795993,0.277153,-0.398975,-0.141946,1.946388,-1.395629
1,1,0,0,0,1,0,0,0,1,1,0,1,0,1,0,0,0,1,0,0,1.510002,0.351551,-0.423744,0.253107,0.388265,-0.147136,-0.365964,-0.901022,-0.471494,-0.825149,-0.007196,2.020670,-0.246594,2.074355,-0.285892,-0.402557,-0.149993,-0.040616,-0.181647,0.105268,-0.296918,-0.087826,-0.046629,0.340748,-0.188041,0.084595,-0.024177,0.125057,-1.006902,-0.068834,-0.333912,0.002983,1.221994,-0.686323,-0.085251,-0.018973,-1.635345,-1.539414,0.720456,0.121236,-0.847759,0.524990,-0.966213,-0.472241,-0.521321,-0.804103,-1.390015,0.136476,-0.025269,-0.054988,-0.237952,-0.795993,0.659437,-0.398975,-0.141946,1.790584,0.545943
2,0,0,0,0,1,0,0,0,1,0,1,1,0,0,0,0,0,0,0,0,-0.639771,0.648573,-0.423744,-0.764979,-0.367561,0.566594,-0.365964,-0.202660,-0.306505,1.888555,-0.007196,1.933926,2.261176,-0.757197,-0.075377,1.715405,-0.380319,-0.040616,-0.181647,0.105268,-0.296918,-0.087826,-0.046629,0.340748,-0.188041,0.084595,-0.024177,0.125057,-0.039453,-0.068834,-0.333912,0.002983,1.221994,-0.585271,-0.085251,-0.018973,1.352527,-0.616370,0.195605,-0.508655,-0.847759,-0.412243,-0.388393,1.488087,-0.521321,0.143087,-0.908949,2.450664,-0.025269,-0.054988,-0.237952,-0.190302,0.103387,-0.398975,-0.141946,0.968290,0.671759
3,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0.053704,-1.722182,-0.423744,3.307367,0.388265,-1.753029,-0.365964,-1.305674,-2.202602,0.362096,-0.007196,0.160809,-0.246594,-0.747537,-0.335357,-0.402557,-0.441812,-0.040616,-0.181647,0.105268,-0.296918,-0.087826,-0.046629,0.340748,-0.188041,0.084595,-0.024177,0.125057,-0.229149,-0.068834,-0.333912,0.002983,0.123387,-0.391283,-0.085251,-0.018973,0.166501,-0.121405,-0.329246,-0.256699,-0.221818,-0.345298,-0.195786,-0.472241,-0.521321,-0.330508,-0.187350,-0.194123,-0.025269,-0.054988,-0.237952,-0.190302,0.659437,-0.398975,-0.141946,-0.220710,1.117834
4,0,0,0,0,0,1,0,0,1,1,0,0,0,1,0,0,0,0,0,0,-0.847814,-0.953899,-0.423744,-0.764979,0.703192,-1.217732,-0.365964,-0.493590,0.755343,-0.909952,-0.007196,-0.823654,-0.246594,-0.757197,0.906318,-0.402557,-1.003996,-0.040616,-0.181647,0.105268,-0.296918,-0.087826,-0.046629,0.340748,-0.188041,0.084595,-0.024177,0.125057,-0.880438,-0.068834,-0.333912,0.002983,0.057196,-0.909248,-0.08

In [35]:
X.dtypes

term_ 60 months                          int64
grade_B                                  int64
grade_F                                  int64
grade_G                                  int64
emp_length_between 5-9 years             int64
emp_length_less than 5 years             int64
home_ownership_OTHERS                    int64
home_ownership_OWN                       int64
home_ownership_RENT                      int64
verification_status_Source Verified      int64
verification_status_Verified             int64
purpose_debt_consolidation               int64
purpose_others                           int64
initial_list_status_w                    int64
application_type_Joint App               int64
hardship_flag_Y                          int64
disbursement_method_DirectPay            int64
debt_settlement_flag_Y                   int64
region_Hawaii-Aleutian                   int64
region_Mountain                          int64
annual_inc                             float64
dti          

In [36]:
y=y.astype(int)

In [37]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=69)
print(f"Training data size: {X_train.shape}")
print(f"Testing data size: {X_test.shape}")

Training data size: (182893, 77)
Testing data size: (78383, 77)


In [38]:
ss=StandardScaler()
X_train_scaled=pd.DataFrame(ss.fit_transform(X_train),columns=X_train.columns)

In [39]:
X_train_scaled=sm.add_constant(X_train_scaled)

In [44]:
X_train_scaled.shape

(182893, 78)

In [45]:
try:
    logreg = sm.Logit(y_train, X_train_scaled).fit(maxiter=100, method='newton')
    print(logreg.summary())
except Exception as e:
    print("Error occurred:", e)

Error occurred: The indices for endog and exog are not aligned


In [50]:
y_train_s=pd.Series(y_train)

In [51]:
y_train_s.head()

235042    0
98725     0
75970     0
233861    1
36732     1
Name: current_loan_status, dtype: int32

In [52]:
X_train.head()

,term_ 60 months,grade_B,grade_F,grade_G,emp_length_between 5-9 years,emp_length_less than 5 years,home_ownership_OTHERS,home_ownership_OWN,home_ownership_RENT,verification_status_Source Verified,verification_status_Verified,purpose_debt_consolidation,purpose_others,initial_list_status_w,application_type_Joint App,hardship_flag_Y,disbursement_method_DirectPay,debt_settlement_flag_Y,region_Hawaii-Aleutian,region_Mountain,annual_inc,dti,delinq_2yrs,inq_last_6mths,mths_since_last_delinq,open_acc,pub_rec,revol_bal,revol_util,total_acc,out_prncp_inv,total_rec_int,total_rec_late_fee,recoveries,last_pymnt_amnt,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,bc_open_to_buy,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_bc_tl,num_il_tl,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,pub_rec_bankruptcies,tax_liens,total_bal_ex_mort,loan_amnt
235042,1,1,0,0,0,0,0,0,1,1,0,0,1,1,0,0,0,0,0,0,-0.327707,-0.067348,0.892104,-0.764979,-0.493532,1.101892,-0.365964,-0.505083,-0.192282,0.446900,-0.007196,-0.840242,-0.246594,-0.757197,2.649114,-0.402557,0.431799,-0.040616,4.281385,0.105268,0.540924,-1.892819,2.047183,0.340748,-0.188041,-0.799333,-1.249215,1.368724,-0.545308,-0.068834,1.182252,2.064590,-0.331070,-0.213984,-0.085251,-0.018973,0.280542,-0.402331,0.326818,-0.508655,-0.847759,-0.345298,-0.388393,1.755404,2.478242,-0.330508,-0.668416,1.624168,-0.025269,-0.054988,2.935668,-0.190302,-0.939205,-0.398975,-0.141946,1.946388,-0.300457
98725,0,1,0,0,0,1,0,0,1,0,1,1,0,0,0,0,0,0,0,0,-0.731657,0.818299,-0.423744,1.271194,0.388265,0.209729,-0.365964,-0.906861,-0.116134,-0.146723,-0.007196,-0.747989,-0.246594,-0.757197,2.048303,-0.402557,-0.767117,-0.040616,-0.181647,0.105268,-0.296918,-0.087826,-0.046629,0.340748,-0.188041,0.084595,-0.024177,0.125057,-1.120719,-0.068834,-0.333912,0.002983,-0.331070,-0.612678,-0.085251,-0.018973,0.006844,-0.335444,0.064393,0.373193,-0.847759,0.056374,-1.158820,1.755404,2.478242,-0.804103,-0.427883,0.632373,-0.025269,-0.054988,-0.237952,-0.795993,-1.854370,-0.398975,-0.141946,-0.412626,-1.055353
75970,0,1,0,0,0,1,0,0,1,1,0,1,0,0,0,0,0,0,0,0,-1.263899,0.513152,-0.423744,1.271194,0.703192,2.529353,-0.365964,-1.066739,-2.015615,1.464539,-0.007196,-0.926340,-0.246594,-0.757197,0.418594,-0.402557,-0.781567,-0.040616,-0.181647,0.105268,-0.296918,-0.087826,-0.046629,0.340748,-0.188041,0.084595,-0.024177,0.125057,0.915349,-0.068834,-0.333912,0.002983,1.610260,2.153373,-0.085251,-0.018973,0.143693,-0.522728,-0.854097,-0.508655,-0.847759,-0.947805,-0.581000,1.755404,-0.521321,0.143087,0.534249,0.962971,-0.025269,-0.054988,-0.237952,2.232466,0.091803,-0.398975,-0.141946,-0.464214,-1.398488
233861,0,0,0,0,0,1,0,0,1,1,0,0,1,1,0,0,0,0,0,0,-0.917161,-1.248212,0.892104,1.271194,-0.745474,-0.860866,-0.365964,-1.004549,-0.615330,-1.418772,-0.007196,-1.131539,-0.246594,-0.757197,-0.764051,-0.402557,-1.059317,-0.040616,-0.181647,0.105268,-0.296918,-0.087826,-0.046629,0.340748,-0.188041,0.084595,-0.024177,0.125057,-1.335708,-0.068834,-0.333912,0.002983,-0.331070,-0.969879,-0.085251,-0.018973,0.166501,-1.726699,-1.247735,-0.886590,-0.847759,-0.077517,-0.966213,1.666298,0.978461,-0.330508,-1.149482,-1.351217,-0.025269,-0.054988,2.935668,-0.795993,0.659437,-0.398975,-0.141946,-1.455818,-1.206905
36732,0,0,0,0,0,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1.197938,-0.757088,-0.423744,-0.764979,0.388265,-0.504001,-0.365964,0.457611,1.461832,-1.333969,-0.007196,-1.032324,-0.246594,-0.757197,-0.527280,-0.402557,-0.793518,-1.525434,-0.181647,-1.411605,0.540924,1.942791,-1.257754,-1.041634,-1.530517,-2.567188,1.977230,1.368724,-0.292381,-0.976552,-0.333912,-2.0586

In [48]:
X_train_scaled.head(20)

,const,term_ 60 months,grade_B,grade_F,grade_G,emp_length_between 5-9 years,emp_length_less than 5 years,home_ownership_OTHERS,home_ownership_OWN,home_ownership_RENT,verification_status_Source Verified,verification_status_Verified,purpose_debt_consolidation,purpose_others,initial_list_status_w,application_type_Joint App,hardship_flag_Y,disbursement_method_DirectPay,debt_settlement_flag_Y,region_Hawaii-Aleutian,region_Mountain,annual_inc,dti,delinq_2yrs,inq_last_6mths,mths_since_last_delinq,open_acc,pub_rec,revol_bal,revol_util,total_acc,out_prncp_inv,total_rec_int,total_rec_late_fee,recoveries,last_pymnt_amnt,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,bc_open_to_buy,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_bc_tl,num_il_tl,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,pub_rec_bankruptcies,tax_liens,total_bal_ex_mort,loan_amnt
0,1.0,1.354841,1.890907,-0.222253,-0.122937,-0.554695,-0.779149,-0.020389,-0.351225,1.109284,1.219476,-0.744248,-1.242349,1.999952,0.848072,-0.144643,-0.002338,-0.068775,-0.327631,-0.069929,-0.254361,-0.327173,-0.066906,0.893529,-0.766331,-0.493828,1.098975,-0.376690,-0.505015,-0.191565,0.446075,-0.007257,-0.838414,-0.246685,-0.757696,2.647822,-0.402463,0.433674,-0.042207,4.267430,0.104621,0.540127,-1.889357,2.044938,0.341210,-0.189172,-0.798293,-1.246184,1.365224,-0.545356,-0.069115,1.184002,2.060554,-0.330932,-0.214109,-0.084484,-0.018474,0.281204,-0.402698,0.328944,-0.507159,-0.846900,-0.343023,-0.387005,1.749086,2.474927,-0.331235,-0.668793,1.624890,-0.026492,-0.05499,2.944660,-0.191309,-0.940004,-0.399810,-0.144382,1.947418,-0.300209
1,1.0,-0.738094,1.890907,-0.222253,-0.122937,-0.554695,1.283451,-0.020389,-0.351225,1.109284,-0.820024,1.343639,0.804927,-0.500012,-1.179145,-0.144643,-0.002338,-0.068775,-0.327631,-0.069929,-0.254361,-0.731354,0.803026,-0.423844,1.269764,0.388130,0.208378,-0.376690,-0.906882,-0.115344,-0.146960,-0.007257,-0.746116,-0.246685,-0.757696,2.047117,-0.402463,-0.765659,-0.042207,-0.183002,0.104621,-0.297407,-0.087543,-0.046661,0.341210,-0.189172,0.084674,-0.023184,0.124907,-1.120877,-0.069115,-0.334455,0.001565,-0.330932,-0.612939,-0.084484,-0.018474,0.007664,-0.335831,0.066295,0.373387,-0.846900,0.059462,-1.158284,1.749086,2.474927,-0.804003,-0.428431,0.632786,-0.026492,-0.05499,-0.237572,-0.797234,-1.856214,-0.399810,-0.144382,-0.412562,-1.054698
2,1.0,-0.738094,1.890907,-0.222253,-0.122937,-0.554695,1.283451,-0.020389,-0.351225,1.109284,1.219476,-0.744248,0.804927,-0.500012,-1.179145,-0.144643,-0.002338,-0.068775,-0.327631,-0.069929,-0.254361,-1.263901,0.503294,-0.423844,1.269764,0.703115,2.523930,-0.376690,-1.066795,-2.016644,1.462706,-0.007257,-0.924553,-0.246685,-0.757696,0.417695,-0.402463,-0.780114,-0.042207,-0.183002,0.104621,-0.297407,-0.087543,-0.046661,0.341210,-0.189172,0.084674,-0.023184,0.124907,0.915582,-0.069115,-0.334455,0.001565,1.610399,2.154059,-0.084484,-0.018474,0.144434,-0.523059,-0.852978,-0.507159,-0.846900,-0.946749,-0.579825,1.749086,-0.521871,0.141532,0.533015,0.963487,-0.026492,-0.05499,-0.237572,2.232391,0.092182,-0.399810,-0.144382,-0.464171,-1.397648
3,1.0,-0.738094,-0.528847,-0.222253,-0.122937,-0.554695,1.283451,-0.020389,-0.351225,1.109284,1.219476,-0.744248,-1.242349,1.999952,0.848072,-0.144643,-0.002338,-0.068775,-0.327631,-0.069929,-0.254361,-0.916965,-1.226815,0.893529,1.269764,-0.745816,-0.860339,-0.376690,-1.004591,-0.615017,-1.417749,-0.007257,-1.129853,-0.246685,-0.757696,-0.764741,-0.402463,-1.057961,-0.042207,-0.183002,0.104621,-0.297407,-0.087543,-0.046661,0.341210,-0.189172,0.084674,-0.023184,0.124907,-1.335906,-0.069115

In [53]:
try:
    logreg = sm.Logit(y_train, X_train).fit(maxiter=100, method='newton')
    print(logreg.summary())
except Exception as e:
    print("Error occurred:", e)

         Current function value: 0.156580
         Iterations: 100


C:\Users\Anmol Gupta\Anaconda3\lib\site-packages\statsmodels\base\model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


                            Logit Regression Results                           
Dep. Variable:     current_loan_status   No. Observations:               182893
Model:                           Logit   Df Residuals:                   182816
Method:                            MLE   Df Model:                           76
Date:                 Wed, 05 Jun 2024   Pseudo R-squ.:                  0.6883
Time:                         13:45:49   Log-Likelihood:                -28637.
converged:                       False   LL-Null:                       -91863.
Covariance Type:             nonrobust   LLR p-value:                     0.000
                                          coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------
term_ 60 months                         2.6697      0.047     57.283      0.000       2.578       2.761
grade_B                                 0.0042  

In [56]:
y_pred = logreg.predict(X_test)

In [57]:
y_pred.head()

218422    0.943585
104572    0.916605
96142     0.999758
216893    0.765621
236653    0.000016
dtype: float64

In [58]:
y_test.head()

218422    1
104572    1
96142     1
216893    1
236653    0
Name: current_loan_status, dtype: int32

In [59]:
y_pred_i=y_pred.astype(int)

In [62]:
y_pred.head()

218422    0.943585
104572    0.916605
96142     0.999758
216893    0.765621
236653    0.000016
dtype: float64

In [63]:
y_pred_r=round(y_pred)

In [64]:
y_pred_r.head()

218422    1.0
104572    1.0
96142     1.0
216893    1.0
236653    0.0
dtype: float64

In [65]:
accuracy_score(y_test,y_pred_r)

0.9414286261051503

In [67]:
X_train2=X_train.copy()

In [68]:
sm.add_constant(X_train2)

KeyboardInterrupt: 

In [69]:
X_train2=sm.add_constant(X_train2)

In [70]:
X_train2.head()

,const,term_ 60 months,grade_B,grade_F,grade_G,emp_length_between 5-9 years,emp_length_less than 5 years,home_ownership_OTHERS,home_ownership_OWN,home_ownership_RENT,verification_status_Source Verified,verification_status_Verified,purpose_debt_consolidation,purpose_others,initial_list_status_w,application_type_Joint App,hardship_flag_Y,disbursement_method_DirectPay,debt_settlement_flag_Y,region_Hawaii-Aleutian,region_Mountain,annual_inc,dti,delinq_2yrs,inq_last_6mths,mths_since_last_delinq,open_acc,pub_rec,revol_bal,revol_util,total_acc,out_prncp_inv,total_rec_int,total_rec_late_fee,recoveries,last_pymnt_amnt,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,bc_open_to_buy,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_bc_tl,num_il_tl,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,pub_rec_bankruptcies,tax_liens,total_bal_ex_mort,loan_amnt
235042,1.0,1,1,0,0,0,0,0,0,1,1,0,0,1,1,0,0,0,0,0,0,-0.327707,-0.067348,0.892104,-0.764979,-0.493532,1.101892,-0.365964,-0.505083,-0.192282,0.446900,-0.007196,-0.840242,-0.246594,-0.757197,2.649114,-0.402557,0.431799,-0.040616,4.281385,0.105268,0.540924,-1.892819,2.047183,0.340748,-0.188041,-0.799333,-1.249215,1.368724,-0.545308,-0.068834,1.182252,2.064590,-0.331070,-0.213984,-0.085251,-0.018973,0.280542,-0.402331,0.326818,-0.508655,-0.847759,-0.345298,-0.388393,1.755404,2.478242,-0.330508,-0.668416,1.624168,-0.025269,-0.054988,2.935668,-0.190302,-0.939205,-0.398975,-0.141946,1.946388,-0.300457
98725,1.0,0,1,0,0,0,1,0,0,1,0,1,1,0,0,0,0,0,0,0,0,-0.731657,0.818299,-0.423744,1.271194,0.388265,0.209729,-0.365964,-0.906861,-0.116134,-0.146723,-0.007196,-0.747989,-0.246594,-0.757197,2.048303,-0.402557,-0.767117,-0.040616,-0.181647,0.105268,-0.296918,-0.087826,-0.046629,0.340748,-0.188041,0.084595,-0.024177,0.125057,-1.120719,-0.068834,-0.333912,0.002983,-0.331070,-0.612678,-0.085251,-0.018973,0.006844,-0.335444,0.064393,0.373193,-0.847759,0.056374,-1.158820,1.755404,2.478242,-0.804103,-0.427883,0.632373,-0.025269,-0.054988,-0.237952,-0.795993,-1.854370,-0.398975,-0.141946,-0.412626,-1.055353
75970,1.0,0,1,0,0,0,1,0,0,1,1,0,1,0,0,0,0,0,0,0,0,-1.263899,0.513152,-0.423744,1.271194,0.703192,2.529353,-0.365964,-1.066739,-2.015615,1.464539,-0.007196,-0.926340,-0.246594,-0.757197,0.418594,-0.402557,-0.781567,-0.040616,-0.181647,0.105268,-0.296918,-0.087826,-0.046629,0.340748,-0.188041,0.084595,-0.024177,0.125057,0.915349,-0.068834,-0.333912,0.002983,1.610260,2.153373,-0.085251,-0.018973,0.143693,-0.522728,-0.854097,-0.508655,-0.847759,-0.947805,-0.581000,1.755404,-0.521321,0.143087,0.534249,0.962971,-0.025269,-0.054988,-0.237952,2.232466,0.091803,-0.398975,-0.141946,-0.464214,-1.398488
233861,1.0,0,0,0,0,0,1,0,0,1,1,0,0,1,1,0,0,0,0,0,0,-0.917161,-1.248212,0.892104,1.271194,-0.745474,-0.860866,-0.365964,-1.004549,-0.615330,-1.418772,-0.007196,-1.131539,-0.246594,-0.757197,-0.764051,-0.402557,-1.059317,-0.040616,-0.181647,0.105268,-0.296918,-0.087826,-0.046629,0.340748,-0.188041,0.084595,-0.024177,0.125057,-1.335708,-0.068834,-0.333912,0.002983,-0.331070,-0.969879,-0.085251,-0.018973,0.166501,-1.726699,-1.247735,-0.886590,-0.847759,-0.077517,-0.966213,1.666298,0.978461,-0.330508,-1.149482,-1.351217,-0.025269,-0.054988,2.935668,-0.795993,0.659437,-0.398975,-0.141946,-1.455818,-1.206905
36732,1.0,0,0,0,0,0,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1.197938,-0.757088,-0.423744,-0.764979,0.388265,-0.504001,-0.365964,0.457611,1.461832,-1.333969,-0.007196,-1.032324,-0.246594,-0.757197,-0.527280,-0.402557,-0.793518,-1.525434,-0.181647,-1.411605,0.540924,1.942791,-1.257754,-1.041634,-1.530517,-2.567188,1.977230,1.368724,-0.292381,-

In [72]:
try:
    logreg_n = sm.Logit(y_train, X_train).fit(maxiter=100, method='newton')
    print(logreg.summary())
except Exception as e:
    print("Error occurred:", e)

         Current function value: 0.156580
         Iterations: 100
                            Logit Regression Results                           
Dep. Variable:     current_loan_status   No. Observations:               182893
Model:                           Logit   Df Residuals:                   182816
Method:                            MLE   Df Model:                           76
Date:                 Wed, 05 Jun 2024   Pseudo R-squ.:                  0.6883
Time:                         14:29:11   Log-Likelihood:                -28637.
converged:                       False   LL-Null:                       -91863.
Covariance Type:             nonrobust   LLR p-value:                     0.000
                                          coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------
term_ 60 months                         2.6697      0.047     57.283      0.000      

C:\Users\Anmol Gupta\Anaconda3\lib\site-packages\statsmodels\base\model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


In [73]:
y_pred_r.head(20)

218422    1.0
104572    1.0
96142     1.0
216893    1.0
236653    0.0
158903    1.0
99008     1.0
59495     0.0
192847    1.0
182687    1.0
59633     1.0
247706    0.0
18857     0.0
170476    1.0
4281      1.0
75913     1.0
142229    1.0
210961    1.0
6065      1.0
8947      1.0
dtype: float64

In [74]:
y_test.head(20)

218422    1
104572    1
96142     1
216893    1
236653    0
158903    1
99008     1
59495     0
192847    1
182687    1
59633     1
247706    0
18857     0
170476    1
4281      1
75913     1
142229    1
210961    1
6065      1
8947      1
Name: current_loan_status, dtype: int32

In [75]:
accuracy_score(y_test,y_pred_r)

0.9414286261051503

In [76]:
log_loss(y_test,y_pred_r)

2.0230193105550383

In [77]:
print(metrics.confusion_matrix(y_test,y_pred_r))

[[12161  3502]
 [ 1089 61631]]


In [81]:
tn, fp, fn, tp = confusion_matrix(y_test, y_pred_r).ravel()

# Calculate sensitivity and specificity
sensitivity = tp / (tp + fn)
specificity = tn / (tn + fp)

# Calculate Youden's Index
youden_index = sensitivity + specificity - 1

print("Sensitivity:", sensitivity)
print("Specificity:", specificity)
print("Youden's Index:", youden_index)

Sensitivity: 0.9826371173469388
Specificity: 0.7764157568792697
Youden's Index: 0.7590528742262084


In [82]:
y_pred_ry = y_pred.apply(lambda x: 0 if x < 0.75 else 1)

print(y_pred_ry.head())

218422    1
104572    1
96142     1
216893    1
236653    0
dtype: int64


In [83]:
accuracy_score(y_test,y_pred_ry)

0.9155684268272457

In [84]:
accuracy_score(y_test,y_pred_i)

0.2338134544480308

In [85]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
lr_model = LogisticRegression(max_iter=1000, random_state=21)
lr_model.fit(X_train,y_train)

LogisticRegression(max_iter=1000, random_state=21)

In [86]:
y_pred_sk = lr_model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred_sk)
report = classification_report(y_test, y_pred_sk)

print(f"Accuracy: {accuracy}")
print("Classification Report:\n", report)

Accuracy: 0.9483051171810214
Classification Report:
               precision    recall  f1-score   support

           0       0.94      0.79      0.86     15663
           1       0.95      0.99      0.97     62720

    accuracy                           0.95     78383
   macro avg       0.95      0.89      0.91     78383
weighted avg       0.95      0.95      0.95     78383



In [90]:
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

In [87]:
result_df=pd.DataFrame(columns=['accuracy','precision','recall','f1-score'])

In [94]:
row={'accuracy': accuracy_score(y_test,y_pred_i), 'precision': precision_score(y_test,y_pred_i), 'recall': recall_score(y_test,y_pred_i),'f1-score':f1_score(y_test,y_pred_i)}

In [96]:
result_df=result_df.append(row,ignore_index=True)

In [97]:
result_df

,accuracy,precision,recall,f1-score
0,0.233813,1.0,0.042474,0.081488


In [100]:
print(metrics.confusion_matrix(y_test,y_pred_i))

[[15663     0]
 [60056  2664]]
